In [4]:
import torch
from DataFidelities.pytorch_radon.radon import Radon,IRadon
import numpy as np
import pandas as pd
from scipy.io import loadmat
from utils.util import addwgn_torch
from math import sqrt
import pickle
import itertools

In [5]:
CTImageMat=loadmat('CT_images_preprocessed.mat')
CTImageArr=CTImageMat['img_cropped'][:,:,0]
CTImageTensor=torch.tensor(CTImageArr,dtype=torch.float).unsqueeze(0).unsqueeze(0)
theta = torch.tensor(np.linspace(0., 180, 360, endpoint=False),dtype=torch.float)
fp=Radon(in_size=512,theta=theta,circle=False)
sino=fp(CTImageTensor)
noisySino=addwgn_torch(sino,50,0,sqrt(2*512**2)).detach().clone()
with open('noisyView360_clamp.pk', 'wb') as f:
    pickle.dump(noisySino.cpu(), f)

In [6]:
fullX=torch.tensor(list(itertools.product(range(noisySino.shape[2]),range(noisySino.shape[3])))).float()
fullX[:,0]=fullX[:,0]/noisySino.shape[2]
fullX[:,1]=fullX[:,1]/noisySino.shape[3]
fully=noisySino.reshape((-1,1))
X=fullX.reshape((-1,360,2))[:,::4,:].reshape((-1,2)).detach().clone()
y=noisySino.squeeze()[:,::4].reshape((-1,1)).detach().clone()

In [10]:
Xcsv=pd.DataFrame(X.numpy())

In [11]:
print(Xcsv)

              0         1
0      0.000000  0.000000
1      0.000000  0.011111
2      0.000000  0.022222
3      0.000000  0.033333
4      0.000000  0.044444
...         ...       ...
65245  0.998621  0.944444
65246  0.998621  0.955556
65247  0.998621  0.966667
65248  0.998621  0.977778
65249  0.998621  0.988889

[65250 rows x 2 columns]


In [12]:
ycsv=pd.DataFrame(y.numpy())

In [14]:
print(ycsv)

              0
0      0.465801
1      0.675858
2      0.264182
3      0.225763
4      0.488038
...         ...
65245  0.000000
65246  0.000000
65247  0.392846
65248  0.000000
65249  0.219304

[65250 rows x 1 columns]


In [15]:
yNormalized=y/sqrt(2*512**2)# (y-min)/(max-min)

In [16]:
yNormalized

tensor([[0.0006],
        [0.0009],
        [0.0004],
        ...,
        [0.0005],
        [0.0000],
        [0.0003]])

In [18]:
yNcsv=pd.DataFrame(yNormalized.numpy())

In [19]:
Xcsv.to_csv('X_new.csv')
ycsv.to_csv('y_new.csv')
yNcsv.to_csv('yN_new.csv')